# Library

In [ ]:
!pip install efficientnet-pytorch -qqq

In [ ]:
from fastai.vision.all import *
import albumentations as A
from efficientnet_pytorch import EfficientNet 

import warnings
warnings.filterwarnings('ignore')

## Set a seed for reproducibility

In [ ]:
set_seed(42)

# CFG

In [ ]:
class CFG:
    size=512
    bs=32
    model='efficientnet-b3'

# Setting up the transforms

In [ ]:
class AlbumentationsTransform(RandTransform):
    '''Transform handler for multiple Albumentation transforms'''
    split_idx, order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
            
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
            
        return PILImage.create(aug_img)

## Training augmentation

In [ ]:
def get_train_aug(): 
    return A.Compose([
        A.RandomResizedCrop(CFG.size, CFG.size),
        A.Transpose(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.HueSaturationValue(
            hue_shift_limit=0.2, 
            sat_shift_limit=0.2, 
            val_shift_limit=0.2, 
            p=0.5
        ),
        A.RandomBrightnessContrast(
            brightness_limit=(-0.1,0.1), 
            contrast_limit=(-0.1, 0.1), 
            p=0.5
        ),
        A.CoarseDropout(p=0.5),
        A.Cutout(p=0.5)
])

## Validation augmentation

In [ ]:
def get_valid_aug(): 
    return A.Compose([
        A.Resize(CFG.size, CFG.size),
        A.CenterCrop(CFG.size, CFG.size, p=1.)
], p=1.)

## Get item transforms for both sets

In [ ]:
item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())

# Get merged data from the 2019 and current competition

In [ ]:
path = Path('/kaggle/input/cassavapreprocessed')

In [ ]:
path.ls()

In [ ]:
train_images = path/'train_images'/'train_images'
test_images  = path/'test_images'/'test_images'

In [ ]:
train_df = pd.read_csv(path/'new_merged.csv', low_memory=False)

In [ ]:
train_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
train_df.head()

# Creating the DataBlock

In [ ]:
def get_x(row): return train_images/row['image_id']
def get_y(row): return row['label']

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                  # Allocate 20% of data to the validation set
                  splitter=RandomSplitter(valid_pct=0.2, seed=42),
                  
                  # Use the functions defined above to get items and labels
                  get_x=get_x,
                  get_y=get_y,
                   
                  # Use our item_tfms on each image seperately 
                  item_tfms=item_tfms,
                   
                  # Normalize a batch of images with imagenet stats
                  batch_tfms=[Normalize.from_stats(*imagenet_stats)])

## Get DataLoaders

In [ ]:
dls = dblock.dataloaders(train_df, bs=CFG.bs)

# Create the pretrained EfficientNet model

In [ ]:
class CassavaModel(Module):
    def __init__(self, num_classes):
        self.effnet = EfficientNet.from_pretrained(CFG.model)
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1536, num_classes)
        
    def forward(self, image):
        batch_size, _, _, _ = image.shape
        
        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        output = self.out(self.dropout(x))
        
        return output

In [ ]:
cassava_net = CassavaModel(dls.c)

# Training the model

We use Cross Entropy with label smoothing as our loss function and Adam as our optimization function. This loss function should be especially helpful here, because the dataset is noisy and it helps with making the model less confident and extreme about it's predictions. The augmentation callback for the model will be CutMix to help with generalization. We have the model train on mixed-precision floating points to speed up the process.

In [ ]:
from fastai.callback.cutmix import *

In [ ]:
learn = Learner(dls, cassava_net, loss_func=LabelSmoothingCrossEntropy(),
               metrics=accuracy, cbs=CutMix()).to_fp16()

## Find the learning rate to fine-tune the head of the model

In [ ]:
learn.lr_find()

## Fine-tune using the Cosine Annealing approach (by using fit_flat_cos with pct_start=0.0). We use 3e-3 as the learning rate. We add a callback to reduce the learning rates after 3 epochs of no improvement in validation loss and a callback to save the model with the best performance during training.

In [ ]:
learn.fit_flat_cos(20, lr=3e-3, pct_start=0.0,
                  cbs=[ReduceLROnPlateau(patience=3),
                      SaveModelCallback()])

# Save and export the model for inference

In [ ]:
learn.save('cassava_net')
learn.export('inference')